In [ ]:
from ipywidgets import *
from scipy import ndimage
from scipy import signal
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update(mpl.rcParamsDefault)
import numpy as np
%matplotlib notebook
from matplotlib.image import imread
from numpy.fft  import fft2, ifft2, fftshift, ifftshift

In [ ]:
#First we will open an image and create a Gaussian PSF, then blur 

def np_fftconvolve(A, B):
    return np.real(fftshift(ifft2(fft2(A)*fft2(B, s=A.shape))))

N=256

im = imread('assets/brain.tif')[0:N,0:N].astype(np.float64)

# Calculate a Gaussian filter whose width depends on the collimator parameters
# L is septa length, d opening, z distance, t thickness
# These are in the same units as the pixel size of the image

L = 4.0
d = 1.0
z = 10.0
t = 0.1

sig = d*(L+z)/L
G = (d**2/(L*(d+t)))**2

x = (N/2)*np.linspace(-1, 1, im.shape[0])
y = (N/2)*np.linspace(-1, 1, im.shape[1])
xx, yy = np.meshgrid(x, y)
psf1 = np.zeros_like(im)
psf1 = (1.0/(sig*np.sqrt(2.0*np.pi))*np.exp(-(xx**2 + yy**2)/(2.0*sig**2))) 
psf1 = psf1/np.sum(psf1) # normalize

#imfilt = signal.convolve2d(im, filt, boundary='symm', mode='same')
imfilt1 = np_fftconvolve(im, psf1)

fig, ax = plt.subplots(2, 3, figsize=(9,6))
fig.suptitle("The imaging equation: blurring", fontsize=16)

# Plot original
ax[0,0].imshow(im, cmap='gray')
ax[0,1].imshow(psf1,cmap='gray')
ax[0,2].imshow(imfilt1,cmap='gray')

ax[0,0].set_title('Original')
ax[0,1].set_title('Gaussian PSF')
ax[0,2].set_title('Blurred')

ax[1,0].plot(im[128,:])
ax[1,1].plot(psf1[128,:])
ax[1,2].plot(imfilt1[128,:])

fig.tight_layout()
fig.subplots_adjust(top=0.88)

Play with the collimator parameters

In [ ]:
# Now let's add noise

# Activity is a collimator indeodent factor reflecting the overall radioactivity level. 
# G is the geomteric activity of the coolimator above

activity = 100.0
noise_level = activity*G
impoisson = np.random.poisson(imfilt1*noise_level+1)

fig, ax = plt.subplots(2, 3, figsize=(9,6))
fig.suptitle("Now with noise", fontsize=16)

# Plot original
ax[0,0].imshow(im, cmap='gray')
ax[0,1].imshow(psf1,cmap='gray')
ax[0,2].imshow(impoisson,cmap='gray')

ax[0,0].set_title('Original')
ax[0,1].set_title('Gaussian PSF')
ax[0,2].set_title('Blurred')

ax[1,0].plot(im[128,:])
ax[1,1].plot(psf1[128,:])
ax[1,2].plot(impoisson[128,:])